Проект ужасно сырой, заранее извиняюсь\
Прогнозы только по прыжкам (одиночным и в каскадах), как к самым дорогим элементам в фигурном катании.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install catboost

In [3]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

In [4]:
# для удовбства визуализации:
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

#снятие ограничения на количество строк
pd.set_option('display.max_rows', None)

In [5]:
import warnings
warnings.filterwarnings('ignore')

### units

In [6]:
units =pd.read_csv('/content/drive/My Drive/Skating_data/units.csv')
units = units.rename(columns={'id':'unit_id'})
units.head()

,unit_id,color,school_id
0,9474,green,244.0
1,733,green,203.0
2,734,green,235.0
3,735,green,168.0
4,736,green,168.0


● id: идентификатор юнита - id спортсмена\
● color: категория\
● school_id: идентификатор школы

In [7]:
units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   unit_id    4596 non-null   int64  
 1   color      4595 non-null   object 
 2   school_id  4007 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 107.8+ KB


In [8]:
units.duplicated().sum()

0

In [9]:
units.duplicated(subset=['unit_id']).sum()

0

In [10]:
units['color'].sort_values().unique()

array(['green', 'lime', nan], dtype=object)

In [11]:
len(units['school_id'].sort_values().unique())

240

240 спортивных школ

### tours

In [12]:
tours =pd.read_csv('/content/drive/My Drive/Skating_data/tournaments.csv')
tours = tours.rename(columns={'id':'tournament_id'})
#del data['Unnamed: 0']
tours.head()

,tournament_id,date_start,date_end,origin_id
0,1,2090-11-29,2090-12-01,2.0
1,2,2091-03-06,2091-03-10,1.0
2,3,2090-10-05,2090-10-08,2.0
3,4,2090-10-18,2090-10-21,2.0
4,5,2090-10-21,2090-10-24,2.0


Турнир состоит из нескольких категорий, оценки по категориям расписаны в total_scores\
● id: идентификатор турнира\
● date_start: дата начала\
● date_end: дата завершения\
● origin_id: место проведения

In [13]:
tours.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tournament_id  142 non-null    int64  
 1   date_start     142 non-null    object 
 2   date_end       142 non-null    object 
 3   origin_id      142 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.6+ KB


In [14]:
tours.duplicated().sum()

0

In [15]:
tours.duplicated(subset=['tournament_id']).sum()

0

### tours_score

In [16]:
tours_score =pd.read_csv('/content/drive/My Drive/Skating_data/tournament_scores.csv')
tours_score = tours_score.rename(columns={'base_score':'base_score_title'})
#del data['Unnamed: 0']
tours_score.head(50)

,id,total_score_id,title,decrease,base_score_title,goe,avg_score
0,1,1,2A,NaN,3.30,0.66,3.96
1,2,1,3F+3Lo,NaN,10.20,-0.11,10.09
2,3,1,3Lz,NaN,0.00,-1.30,5.19
3,4,1,CCoSp4,NaN,3.50,0.56,4.06
4,5,1,FCSp4,NaN,3.20,0.64,3.84
5,6,1,LSp4,NaN,2.70,0.49,3.19
6,7,1,StSq2,NaN,2.60,0.26,2.86
7,8,8,2A,NaN,3.30,0.53,3.83
8,9,8,3F,NaN,0.00,0.42,6.25
9,10,8,3Fq+3Loq,q,10.20,-1.91,8.29


Таблица с оценками поэлементно\
● id: идентификатор оценки за конкретный элемент/комбинацию\
● total_score_id: идентикатор выступления, ключ total_scores.id\
● title: запись элемента или комбинации элементов с отметками об ошибках\
● decrease: за что снижена оценка\
● base_score: базовая оценка (идеал, цена данного элемента/комбинации, сложность)\
● goe: Grade of Execute, качество исполнения, судейские надбавки/убавки\
● avg_score: оценка за элемент/комбинацию (усредненная по судьям)

In [17]:
tours_score['title'] = tours_score['title'].astype(str).apply(lambda col: col + '+')

In [18]:
tours_score.head(10)

,id,total_score_id,title,decrease,base_score_title,goe,avg_score
0,1,1,2A+,NaN,3.3,0.66,3.96
1,2,1,3F+3Lo+,NaN,10.2,-0.11,10.09
2,3,1,3Lz+,NaN,0.0,-1.30,5.19
3,4,1,CCoSp4+,NaN,3.5,0.56,4.06
4,5,1,FCSp4+,NaN,3.2,0.64,3.84
5,6,1,LSp4+,NaN,2.7,0.49,3.19
6,7,1,StSq2+,NaN,2.6,0.26,2.86
7,8,8,2A+,NaN,3.3,0.53,3.83
8,9,8,3F+,NaN,0.0,0.42,6.25
9,10,8,3Fq+3Loq+,q,10.2,-1.91,8.29


In [19]:
pattern_1 = r'([0-4]A[^!q<*e]|[0-4]Lo[^!q<*e]|[0-4]W[^!q<*e]|[0-4]S[^!q<*e]|[0-4]T[^!q<*e]|[0-4]F[^!q<*e]|[0-4]Lz[^!q<*e])'

In [20]:
tours_score['title'][1]

'3F+3Lo+'

In [21]:
result = ' '.join(re.compile(pattern_1).findall(tours_score['title'][1]))
result

'3F+ 3Lo+'

In [22]:
tours_score['re_1'] = 0

In [23]:
%%time
for i in range(172158):
    tours_score['re_1'][i] = ' '.join(re.compile(pattern_1).findall(tours_score['title'][i]))

CPU times: user 4.27 s, sys: 18.6 ms, total: 4.29 s
Wall time: 5.51 s


In [24]:
tours_score.head(30)

,id,total_score_id,title,decrease,base_score_title,goe,avg_score,re_1
0,1,1,2A+,NaN,3.30,0.66,3.96,2A+
1,2,1,3F+3Lo+,NaN,10.20,-0.11,10.09,3F+ 3Lo+
2,3,1,3Lz+,NaN,0.00,-1.30,5.19,3Lz+
3,4,1,CCoSp4+,NaN,3.50,0.56,4.06,
4,5,1,FCSp4+,NaN,3.20,0.64,3.84,
5,6,1,LSp4+,NaN,2.70,0.49,3.19,
6,7,1,StSq2+,NaN,2.60,0.26,2.86,
7,8,8,2A+,NaN,3.30,0.53,3.83,2A+
8,9,8,3F+,NaN,0.00,0.42,6.25,3F+
9,10,8,3Fq+3Loq+,q,10.20,-1.91,8.29,


In [25]:
pattern_2 = r'([0-4]A|[0-4]Lo|[0-4]W|[0-4]S|[0-4]T|[0-4]F|[0-4]Lz)'

In [26]:
tours_score['re_2'] = 0

In [27]:
%%time
for i in range(172158):
    tours_score['re_2'][i] = ''.join(re.compile(pattern_2).findall(tours_score['re_1'][i]))

CPU times: user 4.4 s, sys: 36.3 ms, total: 4.44 s
Wall time: 5.94 s


In [28]:
tours_score.head(30)

,id,total_score_id,title,decrease,base_score_title,goe,avg_score,re_1,re_2
0,1,1,2A+,NaN,3.30,0.66,3.96,2A+,2A
1,2,1,3F+3Lo+,NaN,10.20,-0.11,10.09,3F+ 3Lo+,3F3Lo
2,3,1,3Lz+,NaN,0.00,-1.30,5.19,3Lz+,3Lz
3,4,1,CCoSp4+,NaN,3.50,0.56,4.06,,
4,5,1,FCSp4+,NaN,3.20,0.64,3.84,,
5,6,1,LSp4+,NaN,2.70,0.49,3.19,,
6,7,1,StSq2+,NaN,2.60,0.26,2.86,,
7,8,8,2A+,NaN,3.30,0.53,3.83,2A+,2A
8,9,8,3F+,NaN,0.00,0.42,6.25,3F+,3F
9,10,8,3Fq+3Loq+,q,10.20,-1.91,8.29,,


1W – перекидной прыжок\
1А, 2А, 3А – одинарный, двойной тройной аксель\
1S, 2S, 3S, 4S – одинарный, двойной, тройной, четверной сальхов\
1Lo, 2Lo, 3Lo, 4Lo – одинарный, двойной, тройной, четверной риттбергер\
1Т, 2Т, 3Т, 4Т – одинарный, двойной, тройной, четверной тулуп\
1F, 2F, 3F, 4F – одинарный, двойной, тройной, четверной флип\
1Lz, 2Lz, 3Lz, 4Lz – одинарный, двойной, тройной, четверной лутц\

In [29]:
jumps = ['1A', '2A', '3A', '1S', '2S', '3S', '4S', '1Lo', '2Lo', '3Lo', '4Lo', '1W', '1T', '2T', '3T', '4T', '1F', '2F', '3F', '4F', '1Lz', '2Lz', '3Lz', '4Lz']

In [30]:
for jump in jumps:
    tours_score[jump] = 0
tours_score.head()

,id,total_score_id,title,decrease,base_score_title,goe,avg_score,re_1,re_2,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
0,1,1,2A+,NaN,3.3,0.66,3.96,2A+,2A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,3F+3Lo+,NaN,10.2,-0.11,10.09,3F+ 3Lo+,3F3Lo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1,3Lz+,NaN,0.0,-1.30,5.19,3Lz+,3Lz,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,1,CCoSp4+,NaN,3.5,0.56,4.06,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,1,FCSp4+,NaN,3.2,0.64,3.84,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
del tours_score['re_1']
tours_score.head()

,id,total_score_id,title,decrease,base_score_title,goe,avg_score,re_2,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
0,1,1,2A+,NaN,3.3,0.66,3.96,2A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,3F+3Lo+,NaN,10.2,-0.11,10.09,3F3Lo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1,3Lz+,NaN,0.0,-1.30,5.19,3Lz,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,1,CCoSp4+,NaN,3.5,0.56,4.06,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,1,FCSp4+,NaN,3.2,0.64,3.84,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
jumps[1]

'2A'

In [33]:
for jump in jumps:
    tours_score.loc[(tours_score['re_2'].str.contains(str(jump))), jump]= 1

In [34]:
tours_score.head(30)

,id,total_score_id,title,decrease,base_score_title,goe,avg_score,re_2,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
0,1,1,2A+,NaN,3.30,0.66,3.96,2A,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,3F+3Lo+,NaN,10.20,-0.11,10.09,3F3Lo,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,3,1,3Lz+,NaN,0.00,-1.30,5.19,3Lz,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,4,1,CCoSp4+,NaN,3.50,0.56,4.06,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,1,FCSp4+,NaN,3.20,0.64,3.84,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,6,1,LSp4+,NaN,2.70,0.49,3.19,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,7,1,StSq2+,NaN,2.60,0.26,2.86,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,8,8,2A+,NaN,3.30,0.53,3.83,2A,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,9,8,3F+,NaN,0.00,0.42,6.25,3F,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
9,10,8,3Fq+3Loq+,q,10.20,-1.91,8.29,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
tours_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 32 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                172158 non-null  int64  
 1   total_score_id    172158 non-null  int64  
 2   title             172158 non-null  object 
 3   decrease          41185 non-null   object 
 4   base_score_title  172158 non-null  float64
 5   goe               172158 non-null  float64
 6   avg_score         172158 non-null  float64
 7   re_2              172158 non-null  object 
 8   1A                172158 non-null  int64  
 9   2A                172158 non-null  int64  
 10  3A                172158 non-null  int64  
 11  1S                172158 non-null  int64  
 12  2S                172158 non-null  int64  
 13  3S                172158 non-null  int64  
 14  4S                172158 non-null  int64  
 15  1Lo               172158 non-null  int64  
 16  2Lo               17

In [36]:
tours_score = tours_score.groupby('total_score_id').sum()
tours_score.head()

,id,title,decrease,base_score_title,goe,avg_score,re_2,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
total_score_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,28,2A+3F+3Lo+3Lz+CCoSp4+FCSp4+LSp4+StSq2+,0,25.50,1.20,33.19,2A3F3Lo3Lz,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
8,143,2A+3F+3Fq+3Loq+3Lz<+1Eu+2F+3Lzq+2T+3S+CCoSp4+C...,q<q,35.52,0.35,54.35,2A3F2F2T3S,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0
19,154,2A+3Lo+3Lz+3T+CCoSp4+FCSp4+LSp4+StSq3+,0,20.90,6.68,38.69,2A3Lo3Lz3T,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0
26,378,2A+2A+3T+3F+3Lo+3Lo+2T+3Lz+3Lz+1Eu+3S+CCoSp4+C...,0,35.40,11.79,74.03,2A2A3T3F3Lo3Lo2T3Lz3Lz3S,0,2,0,0,0,1,0,0,0,2,0,0,0,1,1,0,0,0,1,0,0,0,2,0
38,287,2A+3F+3Lz+3Lo+CCoSp3+FSSp4+LSp4+StSq3+,0,26.10,2.95,34.88,2A3F3Lz3Lo,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


Удаляем все суммарные оценки, так как они могут приводить к утечке в будущем (другой возможный вариант с таргетом в виде оценок), оставляем болько базовую сумму оценок. может пригодиться.

In [37]:
del tours_score['id']
del tours_score['decrease']
del tours_score['goe']
del tours_score['avg_score']
tours_score.head()

,title,base_score_title,re_2,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
total_score_id,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2A+3F+3Lo+3Lz+CCoSp4+FCSp4+LSp4+StSq2+,25.50,2A3F3Lo3Lz,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
8,2A+3F+3Fq+3Loq+3Lz<+1Eu+2F+3Lzq+2T+3S+CCoSp4+C...,35.52,2A3F2F2T3S,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0
19,2A+3Lo+3Lz+3T+CCoSp4+FCSp4+LSp4+StSq3+,20.90,2A3Lo3Lz3T,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0
26,2A+2A+3T+3F+3Lo+3Lo+2T+3Lz+3Lz+1Eu+3S+CCoSp4+C...,35.40,2A2A3T3F3Lo3Lo2T3Lz3Lz3S,0,2,0,0,0,1,0,0,0,2,0,0,0,1,1,0,0,0,1,0,0,0,2,0
38,2A+3F+3Lz+3Lo+CCoSp3+FSSp4+LSp4+StSq3+,26.10,2A3F3Lz3Lo,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [38]:
tours_score.describe()

,base_score_title,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
count,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000,21301.000000
mean,20.805893,0.369278,0.631238,0.060514,0.010375,0.199803,0.244636,0.011690,0.087320,0.310126,0.260316,0.001737,0.000141,0.007699,0.556312,0.275386,0.021501,0.037510,0.286043,0.241022,0.000704,0.085160,0.349702,0.374865,0.004037
std,10.067563,0.631090,0.725212,0.280425,0.106746,0.469724,0.502585,0.120268,0.314858,0.574469,0.508959,0.041642,0.011867,0.093130,0.709748,0.501481,0.170356,0.213294,0.580033,0.525348,0.026528,0.294194,0.641670,0.649366,0.071092
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.390000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,19.350000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,26.400000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,82.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,4.000000,2.000000,2.000000


В столбце mean в таблице выше это % выступлений на соревнованиях, где были абсолютно чисто исполнены прыжки из списка. Нужная информация для представления полной картины по прыжкам


In [39]:
tours_score = tours_score.reset_index('total_score_id')

In [40]:
tours_score.head()

,total_score_id,title,base_score_title,re_2,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
0,1,2A+3F+3Lo+3Lz+CCoSp4+FCSp4+LSp4+StSq2+,25.50,2A3F3Lo3Lz,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,8,2A+3F+3Fq+3Loq+3Lz<+1Eu+2F+3Lzq+2T+3S+CCoSp4+C...,35.52,2A3F2F2T3S,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0
2,19,2A+3Lo+3Lz+3T+CCoSp4+FCSp4+LSp4+StSq3+,20.90,2A3Lo3Lz3T,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0
3,26,2A+2A+3T+3F+3Lo+3Lo+2T+3Lz+3Lz+1Eu+3S+CCoSp4+C...,35.40,2A2A3T3F3Lo3Lo2T3Lz3Lz3S,0,2,0,0,0,1,0,0,0,2,0,0,0,1,1,0,0,0,1,0,0,0,2,0
4,38,2A+3F+3Lz+3Lo+CCoSp3+FSSp4+LSp4+StSq3+,26.10,2A3F3Lz3Lo,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [41]:
tours_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_score_id    21301 non-null  int64  
 1   title             21301 non-null  object 
 2   base_score_title  21301 non-null  float64
 3   re_2              21301 non-null  object 
 4   1A                21301 non-null  int64  
 5   2A                21301 non-null  int64  
 6   3A                21301 non-null  int64  
 7   1S                21301 non-null  int64  
 8   2S                21301 non-null  int64  
 9   3S                21301 non-null  int64  
 10  4S                21301 non-null  int64  
 11  1Lo               21301 non-null  int64  
 12  2Lo               21301 non-null  int64  
 13  3Lo               21301 non-null  int64  
 14  4Lo               21301 non-null  int64  
 15  1W                21301 non-null  int64  
 16  1T                21301 non-null  int64 

Больше 21 тыс. уникальных выступлений

Разные пометки ошибок

In [42]:
#tours_score['sum_score'] = tours_score['base_score'] + tours_score['goe']
#tours_score.head()

In [43]:
#tours_score.query("sum_score!=avg_score").head()

In [44]:
#tours_score.query("sum_score!=avg_score")['sum_score'].count()

In [45]:
#tours_score.loc[~(tours_score['avg_score'] == tours_score['sum_score'])].head()

In [46]:
#tours_score.query('id == 6')['avg_score']

In [47]:
#type(tours_score.query('id == 6')['avg_score'])

In [48]:
#tours_score.query('id == 6')['sum_score'] == 3,19

In [49]:
#tours_score.query('id == 6')['avg_score'] + tours_score.query('id == 6')['sum_score']

In [50]:
#tours_score.query('id == 6')['sum_score']

In [51]:
#tours_score.query("avg_score==sum_score").head()

In [52]:
#tours_score.query("avg_score==sum_score")['sum_score'].count()

In [53]:
tours_score.duplicated().sum()

0

In [54]:
tours_score.duplicated(subset=['total_score_id']).sum()

0

### total_scores

In [55]:
total_scores =pd.read_csv('/content/drive/My Drive/Skating_data/total_scores.csv')
#del data['Unnamed: 0']
total_scores = total_scores.rename(columns={'id':'total_score_id'})
total_scores = total_scores.rename(columns={'base_score':'base_score_perf'})
total_scores.head(10)

,total_score_id,unit_id,tournament_id,base_score_perf,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
0,442027,304,4785,47.20,43.47,102.70,59.23,0.0,17,1,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,293.74,2
1,442028,604,4785,45.40,46.71,101.19,54.48,0.0,18,2,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75,1
2,442029,409,4785,44.00,46.82,99.20,52.38,0.0,6,3,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57,4
3,442030,524,4785,46.20,44.86,91.84,47.98,-1.0,3,4,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,3,292.42,3
4,442031,412,4785,44.20,42.59,89.73,48.14,-1.0,15,5,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,6,265.34,6
5,442032,2748,4785,44.97,43.84,88.19,45.35,-1.0,5,6,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,7,255.48,7
6,442034,416,4785,41.81,42.82,86.97,45.15,-1.0,9,8,Короткая программа,x Надбавка за прыжки во второй половине програ...,5,265.63,5
7,442035,605,4785,41.61,41.93,83.99,43.06,-1.0,14,9,Короткая программа,q Прыжок приземлён в четверть ! Неясное ребро ...,9,249.51,9
8,442036,413,4785,44.30,39.19,81.47,43.28,-1.0,13,10,Короткая программа,< Недокрученный прыжок ! Неясное ребро на толч...,13,237.98,13
9,442037,521,4785,42.07,43.60,80.97,38.37,-1.0,11,11,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,10,249.08,10


Оценки за выступления по категориям и общие за турнир\
● id: идентификатор выступления, джойнится с tournament_scores.total_score_id\
● unit_id: идентификатор юнита, ключ к units.id\
● tournament_id: идентификатор турнира, tournaments.id\
● components_score: артистизм (мастерство, композиция, хореография)\
● base_score: базовая оценка за элементы в выступлении (идеал)\
● elements_score: реальная оценка всех выполненных элементов, base_score+goe\
● decreasings_score: отдельный вычет(понижение) оценки. например за тайминг музыки, падение одежды и т.д.\
● total_score: components_score+elements_score+decreasings_score за выступление\
● starting_place: номер в жеребьевке\
● place: занятое место в категории category_name+segment_name\
● segment_name: название сегмента\
● info: комментарии и пояснения к оценке\
● overall_place: итоговое место в турнире\
● overall_total_score: итоговая оценка за весь турнир\
● overall_place_str: комментарии, пояснения\

In [56]:
total_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   total_score_id       21301 non-null  int64  
 1   unit_id              21301 non-null  int64  
 2   tournament_id        21301 non-null  int64  
 3   base_score_perf      21301 non-null  float64
 4   components_score     21301 non-null  float64
 5   total_score          21301 non-null  float64
 6   elements_score       21301 non-null  float64
 7   decreasings_score    21301 non-null  float64
 8   starting_place       21301 non-null  int64  
 9   place                21301 non-null  int64  
 10  segment_name         21284 non-null  object 
 11  info                 20720 non-null  object 
 12  overall_place        21301 non-null  int64  
 13  overall_total_score  21284 non-null  float64
 14  overall_place_str    10814 non-null  object 
dtypes: float64(6), int64(6), object(3)
m

In [57]:
total_scores.duplicated().sum()

0

In [58]:
total_scores.duplicated(subset=['total_score_id']).sum()

0

In [59]:
total_scores['segment_name'].sort_values().unique()

array(['Kopoткaя пpoгpaммa', 'Интерпретация', 'Интерпретация.',
       'Кopoткaя пpoгpaммa', 'Кoроткая программа', 'Коpоткая программа',
       'Короткая программа', 'Пpoизвoльнaя пpoгpaммa',
       'Пpoизвольнaя программа', 'Пpoизвольная программа',
       'Прoизвольная прoграмма', 'Произвольнaя программа',
       'Произвольная программа', 'Произвольная программа.', 'Прыжки',
       'Элeмeнты', 'Элементы', nan], dtype=object)

In [60]:
len(total_scores['info'].sort_values().unique())

1211

1211 уникальных значений надбавок и пояснений за ошибки

In [61]:
total_scores['info'].sort_values().unique()[:100]

array(['! Неясное ребро на толчке F/Lz',
       '! Неясное ребро на толчке F/Lz F Падение в элементе',
       "! Неясное ребро на толчке F/Lz F Падение в элементе nS Нет базовой позиции 'волчок'",
       "! Неясное ребро на толчке F/Lz REP Повторение прыжка не в каскаде F Падение в элементе nC Нет базовой позиции 'либела'",
       '! Неясное ребро на толчке F/Lz REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй половине программы (10%)',
       '! Неясное ребро на толчке F/Lz REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй половине программы (10%) B Элемент получил бонус@@@Бонусных баллов: +1,00',
       '! Неясное ребро на толчке F/Lz REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе',
       '! Неясное ребро на толчке F/Lz REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе B Элемент получил бонус@@@Бонусных баллов: +1,00',
      

In [62]:
total_scores['overall_place_str'].sort_values().unique()

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
       '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39',
       '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49',
       '5', '50', '6', '7', '8', '9', 'FNR', 'NQD', 'WD', nan],
      dtype=object)

In [63]:
total_scores['total_score_id'].sort_values().unique()

array([     1,      8,     19, ..., 463637, 463638, 463639])

## Объединение таблиц

In [64]:
%%time
data = total_scores.merge(units, on='unit_id', how='left')
data.head()

CPU times: user 9.65 ms, sys: 41 µs, total: 9.69 ms
Wall time: 12.2 ms


,total_score_id,unit_id,tournament_id,base_score_perf,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id
0,442027,304,4785,47.2,43.47,102.70,59.23,0.0,17,1,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,293.74,2,lime,62.0
1,442028,604,4785,45.4,46.71,101.19,54.48,0.0,18,2,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75,1,lime,62.0
2,442029,409,4785,44.0,46.82,99.20,52.38,0.0,6,3,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57,4,lime,198.0
3,442030,524,4785,46.2,44.86,91.84,47.98,-1.0,3,4,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,3,292.42,3,lime,105.0
4,442031,412,4785,44.2,42.59,89.73,48.14,-1.0,15,5,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,6,265.34,6,lime,198.0


In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   total_score_id       21301 non-null  int64  
 1   unit_id              21301 non-null  int64  
 2   tournament_id        21301 non-null  int64  
 3   base_score_perf      21301 non-null  float64
 4   components_score     21301 non-null  float64
 5   total_score          21301 non-null  float64
 6   elements_score       21301 non-null  float64
 7   decreasings_score    21301 non-null  float64
 8   starting_place       21301 non-null  int64  
 9   place                21301 non-null  int64  
 10  segment_name         21284 non-null  object 
 11  info                 20720 non-null  object 
 12  overall_place        21301 non-null  int64  
 13  overall_total_score  21284 non-null  float64
 14  overall_place_str    10814 non-null  object 
 15  color                21301 non-null 

In [66]:
%%time
data = data.merge(tours, on='tournament_id', how='left')
data.head()

CPU times: user 9.22 ms, sys: 0 ns, total: 9.22 ms
Wall time: 9.35 ms


,total_score_id,unit_id,tournament_id,base_score_perf,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id
0,442027,304,4785,47.2,43.47,102.70,59.23,0.0,17,1,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,293.74,2,lime,62.0,2091-12-20,2091-12-24,2.0
1,442028,604,4785,45.4,46.71,101.19,54.48,0.0,18,2,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75,1,lime,62.0,2091-12-20,2091-12-24,2.0
2,442029,409,4785,44.0,46.82,99.20,52.38,0.0,6,3,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57,4,lime,198.0,2091-12-20,2091-12-24,2.0
3,442030,524,4785,46.2,44.86,91.84,47.98,-1.0,3,4,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,3,292.42,3,lime,105.0,2091-12-20,2091-12-24,2.0
4,442031,412,4785,44.2,42.59,89.73,48.14,-1.0,15,5,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,6,265.34,6,lime,198.0,2091-12-20,2091-12-24,2.0


In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   total_score_id       21301 non-null  int64  
 1   unit_id              21301 non-null  int64  
 2   tournament_id        21301 non-null  int64  
 3   base_score_perf      21301 non-null  float64
 4   components_score     21301 non-null  float64
 5   total_score          21301 non-null  float64
 6   elements_score       21301 non-null  float64
 7   decreasings_score    21301 non-null  float64
 8   starting_place       21301 non-null  int64  
 9   place                21301 non-null  int64  
 10  segment_name         21284 non-null  object 
 11  info                 20720 non-null  object 
 12  overall_place        21301 non-null  int64  
 13  overall_total_score  21284 non-null  float64
 14  overall_place_str    10814 non-null  object 
 15  color                21301 non-null 

In [68]:
%%time
data = data.merge(tours_score, on='total_score_id', how='left')
data.head(3)

CPU times: user 22.5 ms, sys: 12.9 ms, total: 35.5 ms
Wall time: 37.6 ms


,total_score_id,unit_id,tournament_id,base_score_perf,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,title,base_score_title,re_2,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
0,442027,304,4785,47.2,43.47,102.70,59.23,0.0,17,1,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,293.74,2,lime,62.0,2091-12-20,2091-12-24,2.0,4Lz+4S+3T+FCSp4+3A+StSq3+CCoSp4+CSSp4+,38.4,4Lz4S3T3A,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,442028,604,4785,45.4,46.71,101.19,54.48,0.0,18,2,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75,1,lime,62.0,2091-12-20,2091-12-24,2.0,4T+3T+4Sq+CCoSp4+3A+CSSp3+StSq4+FCSp4+,36.6,4T3T3A,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
2,442029,409,4785,44.0,46.82,99.20,52.38,0.0,6,3,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57,4,lime,198.0,2091-12-20,2091-12-24,2.0,4T+3T+4S+FCSp4+3Aq+CCoSp3+StSq3+CSSp2+,35.2,4T3T4S,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0


In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 47 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   total_score_id       21301 non-null  int64  
 1   unit_id              21301 non-null  int64  
 2   tournament_id        21301 non-null  int64  
 3   base_score_perf      21301 non-null  float64
 4   components_score     21301 non-null  float64
 5   total_score          21301 non-null  float64
 6   elements_score       21301 non-null  float64
 7   decreasings_score    21301 non-null  float64
 8   starting_place       21301 non-null  int64  
 9   place                21301 non-null  int64  
 10  segment_name         21284 non-null  object 
 11  info                 20720 non-null  object 
 12  overall_place        21301 non-null  int64  
 13  overall_total_score  21284 non-null  float64
 14  overall_place_str    10814 non-null  object 
 15  color                21301 non-null 

Какие признаки и таргет возьмем в датасет:\
● id: идентификатор выступления - не берем\
● unit_id: идентификатор спортсмена - берем, чтобы далее сортировать и предсказывать по спортсмену?\
● tournament_id: идентификатор турнира- не берем\
● base_score_perf: базовая оценка за элементы в выступлении (идеал) - берем (это та техническая оценка за все элементы, на которую претендуем спортсмен.\
● components_score: артистизм (мастерство, композиция, хореография) - не берем, утечка данных \
● total_score: components_score+elements_score+decreasings_score за выступление - не берем утечка данных\
● elements_score: реальная оценка всех выполненных элементов, base_score+goe - не берем утечка данных\
● decreasings_score: отдельный вычет (понижение) оценки. например за тайминг музыки, падение одежды и т.д. - не берем утечка данных\
● starting_place: номер в жеребьевке - берем, может повлиять на результат теоритически\
● place: занятое место в категории category_name+segment_name - не берем, утечка данных \
● segment_name: название сегмента (вид программы) - берем\
● info: комментарии и пояснения к оценке - не берем, утечка данных \
● overall_place: итоговое место в турнире - не берем, утечка данных \
● overall_total_score: итоговая оценка за весь турнир - не берем, утечка данных\
● overall_place_str: комментарии, пояснения - не берем, утечка данных \
● color   категория спортсмена - берем\

● school_id идентификатор школы - берем \
● date_start -  дата начала соревнования - не берем\          
● date_end - дата завершения соревнования - берем. (вычислим актуальная дата минус время окончания соревнования, так что будет признак как давно спортсмен прыгал чисто свои прыжки).\

● origin_id -место проведения - не берем\   
● title  - вся программа спорсмена на данном соревновании - не берем\
● base_score_title  базовая оценка (идеал, цена данного элемента/комбинации, сложность) - это суммарная оценка за все элементы из таблицы tours_score, а там за все каскады стоят нули. Это ошибка и неточность, и у нас в принципе есть признак base_score_perf, что то же самое по сути. Так что это признак - не берем.\
● re_2 - элементы программы, выполненные чисто - не берем (это идет в таргет в виде столбцов прыжков)                





Формируем итоговый датасет:

In [70]:
data = data[['unit_id', 'base_score_perf', 'starting_place', 'segment_name', 'color', 'school_id', 'date_end',\
             '1A', '2A', '3A', '1S', '2S', '3S', '4S', '1Lo', '2Lo', '3Lo', '4Lo', '1W', '1T', '2T', '3T', '4T', '1F', '2F', '3F', '4F', '1Lz', '2Lz', '3Lz', '4Lz']]
data.head()

,unit_id,base_score_perf,starting_place,segment_name,color,school_id,date_end,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
0,304,47.2,17,Короткая программа,lime,62.0,2091-12-24,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,604,45.4,18,Короткая программа,lime,62.0,2091-12-24,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
2,409,44.0,6,Короткая программа,lime,198.0,2091-12-24,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
3,524,46.2,3,Короткая программа,lime,105.0,2091-12-24,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,412,44.2,15,Короткая программа,lime,198.0,2091-12-24,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0


In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   unit_id          21301 non-null  int64  
 1   base_score_perf  21301 non-null  float64
 2   starting_place   21301 non-null  int64  
 3   segment_name     21284 non-null  object 
 4   color            21301 non-null  object 
 5   school_id        20384 non-null  float64
 6   date_end         21301 non-null  object 
 7   1A               21301 non-null  int64  
 8   2A               21301 non-null  int64  
 9   3A               21301 non-null  int64  
 10  1S               21301 non-null  int64  
 11  2S               21301 non-null  int64  
 12  3S               21301 non-null  int64  
 13  4S               21301 non-null  int64  
 14  1Lo              21301 non-null  int64  
 15  2Lo              21301 non-null  int64  
 16  3Lo              21301 non-null  int64  
 17  4Lo         

In [72]:
len(data['unit_id'].sort_values().unique())

3393

3393 спортсмена в датасете

In [73]:
len(data['starting_place'].sort_values().unique())

55

55 уникальных номеров в жеребьевке

In [74]:
data['segment_name'].sort_values().unique()

array(['Kopoткaя пpoгpaммa', 'Интерпретация', 'Интерпретация.',
       'Кopoткaя пpoгpaммa', 'Кoроткая программа', 'Коpоткая программа',
       'Короткая программа', 'Пpoизвoльнaя пpoгpaммa',
       'Пpoизвольнaя программа', 'Пpoизвольная программа',
       'Прoизвольная прoграмма', 'Произвольнaя программа',
       'Произвольная программа', 'Произвольная программа.', 'Прыжки',
       'Элeмeнты', 'Элементы', nan], dtype=object)

In [75]:
# очистим все опечатки в столбце segment_name:
data['segment_name'] = data['segment_name'].str.replace('Kopoткaя пpoгpaммa', 'Короткая программа')
data['segment_name'] = data['segment_name'].str.replace('Кopoткaя пpoгpaммa', 'Короткая программа')
data['segment_name'] = data['segment_name'].str.replace('Кoроткая программа', 'Короткая программа')
data['segment_name'] = data['segment_name'].str.replace('Коpоткая программа', 'Короткая программа')
data['segment_name'] = data['segment_name'].str.replace('Короткая программа', 'Короткая программа')

data['segment_name'] = data['segment_name'].str.replace('Пpoизвoльнaя пpoгpaммa', 'Произвольная программа')
data['segment_name'] = data['segment_name'].str.replace('Пpoизвольнaя программа', 'Произвольная программа')
data['segment_name'] = data['segment_name'].str.replace('Пpoизвольная программа', 'Произвольная программа')
data['segment_name'] = data['segment_name'].str.replace('Прoизвольная прoграмма', 'Произвольная программа')
data['segment_name'] = data['segment_name'].str.replace('Произвольнaя программа', 'Произвольная программа')
data['segment_name'] = data['segment_name'].str.replace('Произвольная программа', 'Произвольная программа')
data['segment_name'] = data['segment_name'].str.replace('Произвольная программа.', 'Произвольная программа')

data['segment_name'] = data['segment_name'].str.replace('Интерпретация', 'Интерпретация')
data['segment_name'] = data['segment_name'].str.replace('Интерпретация.', 'Интерпретация')

data['segment_name'] = data['segment_name'].str.replace('Элeмeнты', 'Элементы')
data['segment_name'] = data['segment_name'].str.replace('Элементы', 'Элементы')

In [76]:
data['segment_name'].sort_values().unique()

array(['Интерпретация', 'Короткая программа', 'Произвольная программа',
       'Прыжки', 'Элементы', nan], dtype=object)

In [77]:
data['segment_name'].value_counts()

segment_name
Произвольная программа    12192
Короткая программа         8158
Элементы                    659
Прыжки                      254
Интерпретация                21
Name: count, dtype: int64

In [78]:
# заполняем пропуски спортсменов без программы на 0 (их мало)
data['segment_name'] = data['segment_name'].fillna(0)

In [79]:
data['segment_name'] = data['segment_name'].astype('category')

In [80]:
len(data['color'].sort_values().unique())

2

In [81]:
data['color'].value_counts()

color
green    15521
lime      5780
Name: count, dtype: int64

2 разных  категории спортсменов

In [82]:
data['color'] = data['color'].astype('category')

In [83]:
len(data['school_id'].sort_values().unique())

240

240 разных школ спортсменов

In [84]:
# заполняем пропуски школ спортсменов на 0 (их мало)
data['school_id'] = data['school_id'].fillna(0)

In [85]:
data['school_id'] = data['school_id'].astype('category')

In [86]:
# data['date_end'] = pd.to_datetime(data['date_end'], format='%Y-%m-%dT')
data['date_end'] = data['date_end'].apply(lambda x: pd.to_datetime(x, format = 'mixed', yearfirst = True))

In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   unit_id          21301 non-null  int64         
 1   base_score_perf  21301 non-null  float64       
 2   starting_place   21301 non-null  int64         
 3   segment_name     21301 non-null  category      
 4   color            21301 non-null  category      
 5   school_id        21301 non-null  category      
 6   date_end         21301 non-null  datetime64[ns]
 7   1A               21301 non-null  int64         
 8   2A               21301 non-null  int64         
 9   3A               21301 non-null  int64         
 10  1S               21301 non-null  int64         
 11  2S               21301 non-null  int64         
 12  3S               21301 non-null  int64         
 13  4S               21301 non-null  int64         
 14  1Lo              21301 non-null  int64

In [88]:
from datetime import datetime
datetime.now()

datetime.datetime(2024, 6, 8, 0, 5, 25, 140502)

In [89]:
data['days_of_last_tour'] = (data['date_end'] - datetime.now()).dt.days

In [90]:
data.head()

,unit_id,base_score_perf,starting_place,segment_name,color,school_id,date_end,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,days_of_last_tour
0,304,47.2,17,Короткая программа,lime,62.0,2091-12-24,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,24669
1,604,45.4,18,Короткая программа,lime,62.0,2091-12-24,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,24669
2,409,44.0,6,Короткая программа,lime,198.0,2091-12-24,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,24669
3,524,46.2,3,Короткая программа,lime,105.0,2091-12-24,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,24669
4,412,44.2,15,Короткая программа,lime,198.0,2091-12-24,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,24669


In [91]:
data = data.sort_values(by='date_end', ascending=True)

In [92]:
data.head()

,unit_id,base_score_perf,starting_place,segment_name,color,school_id,date_end,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,days_of_last_tour
2850,906,49.24,11,Произвольная программа,lime,62.0,2089-09-23,0,2,0,0,1,0,0,0,0,2,0,0,0,2,1,0,0,0,0,0,0,0,0,0,23847
2842,334,57.68,10,Произвольная программа,lime,62.0,2089-09-23,0,2,0,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,2,0,23847
2844,5095,65.03,20,Произвольная программа,lime,62.0,2089-09-23,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,1,1,0,1,0,0,0,2,0,23847
2845,2336,55.85,18,Произвольная программа,lime,62.0,2089-09-23,0,2,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,2,0,0,0,1,0,23847
2846,333,54.12,7,Произвольная программа,lime,62.0,2089-09-23,0,2,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,2,0,23847


In [93]:
# del data['date_end']

In [94]:
data = data.set_index('date_end')
data.head()

,unit_id,base_score_perf,starting_place,segment_name,color,school_id,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,days_of_last_tour
date_end,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2089-09-23,906,49.24,11,Произвольная программа,lime,62.0,0,2,0,0,1,0,0,0,0,2,0,0,0,2,1,0,0,0,0,0,0,0,0,0,23847
2089-09-23,334,57.68,10,Произвольная программа,lime,62.0,0,2,0,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,2,0,23847
2089-09-23,5095,65.03,20,Произвольная программа,lime,62.0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,1,1,0,1,0,0,0,2,0,23847
2089-09-23,2336,55.85,18,Произвольная программа,lime,62.0,0,2,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,2,0,0,0,1,0,23847
2089-09-23,333,54.12,7,Произвольная программа,lime,62.0,0,2,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,2,0,23847


In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21301 entries, 2089-09-23 to 2092-05-03
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   unit_id            21301 non-null  int64   
 1   base_score_perf    21301 non-null  float64 
 2   starting_place     21301 non-null  int64   
 3   segment_name       21301 non-null  category
 4   color              21301 non-null  category
 5   school_id          21301 non-null  category
 6   1A                 21301 non-null  int64   
 7   2A                 21301 non-null  int64   
 8   3A                 21301 non-null  int64   
 9   1S                 21301 non-null  int64   
 10  2S                 21301 non-null  int64   
 11  3S                 21301 non-null  int64   
 12  4S                 21301 non-null  int64   
 13  1Lo                21301 non-null  int64   
 14  2Lo                21301 non-null  int64   
 15  3Lo                21301 non-null  i

Получили готовый датасет для обучения.

Я не совсем поняла, модель надо обучать по каждому отдельному спортсмену? Если так, то надо будет столько обучений и моделей, сколько у нас есть уникальных спорсменов (и их выступлений). \
Как это все потом собрать в отдельную большую модель, которая зная всех спортсменов сможет предсказывать любого следующего? Этого я совсем не понимаю.

## ПРИМЕР:

Для примера построила модель для одного спортсмена, используя модель Catboost:

Разбиваем на выборки:

In [96]:
data[(data["unit_id"] == 906)]

,unit_id,base_score_perf,starting_place,segment_name,color,school_id,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,days_of_last_tour
date_end,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2089-09-23,906,49.24,11,Произвольная программа,lime,62.0,0,2,0,0,1,0,0,0,0,2,0,0,0,2,1,0,0,0,0,0,0,0,0,0,23847
2089-09-23,906,28.43,5,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,23847
2089-10-06,906,42.27,3,Произвольная программа,lime,62.0,0,2,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,23860
2089-10-06,906,24.89,13,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23860
2090-02-18,906,27.33,9,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,23995
2090-02-18,906,53.51,1,Произвольная программа,lime,62.0,0,2,0,0,0,1,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,2,0,23995
2090-02-18,906,46.18,13,Произвольная программа,lime,62.0,0,2,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,2,0,23995
2090-02-18,906,32.23,11,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,23995
2090-03-30,906,56.47,9,Произвольная программа,lime,62.0,0,2,0,0,0,1,0,0,0,2,0,0,0,1,1,0,0,0,0,0,0,0,2,0,24035


In [97]:
len(data[(data["unit_id"] == 906)])

29

In [98]:
# Задаем спорсмена для примера
unit_id = 906

# Выделяем только те данные, которые относятся к данному спортсмену
data_unit = data[(data["unit_id"] == unit_id)]

del data_unit['unit_id'] # мы уже использовали фильтр по определенному спортсмену, этот признак больше не нужен
del data_unit['starting_place'] # эти данные тренеру до соревнования неизвестны, смысла находит от этого признака зависимость нет
data_unit['segment_name'] = data_unit['segment_name'].astype('str') # чтобы сработал catboost
data_unit['color'] = data_unit['color'].astype('str') # чтобы сработал catboost
data_unit['school_id'] = data_unit['school_id'].astype('str') # чтобы сработал catboost
data_unit.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29 entries, 2089-09-23 to 2092-03-01
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   base_score_perf    29 non-null     float64
 1   segment_name       29 non-null     object 
 2   color              29 non-null     object 
 3   school_id          29 non-null     object 
 4   1A                 29 non-null     int64  
 5   2A                 29 non-null     int64  
 6   3A                 29 non-null     int64  
 7   1S                 29 non-null     int64  
 8   2S                 29 non-null     int64  
 9   3S                 29 non-null     int64  
 10  4S                 29 non-null     int64  
 11  1Lo                29 non-null     int64  
 12  2Lo                29 non-null     int64  
 13  3Lo                29 non-null     int64  
 14  4Lo                29 non-null     int64  
 15  1W                 29 non-null     int64  
 16  1T      

In [99]:
data_unit

,base_score_perf,segment_name,color,school_id,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,days_of_last_tour
date_end,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2089-09-23,49.24,Произвольная программа,lime,62.0,0,2,0,0,1,0,0,0,0,2,0,0,0,2,1,0,0,0,0,0,0,0,0,0,23847
2089-09-23,28.43,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,23847
2089-10-06,42.27,Произвольная программа,lime,62.0,0,2,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,23860
2089-10-06,24.89,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23860
2090-02-18,27.33,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,23995
2090-02-18,53.51,Произвольная программа,lime,62.0,0,2,0,0,0,1,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,2,0,23995
2090-02-18,46.18,Произвольная программа,lime,62.0,0,2,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,2,0,23995
2090-02-18,32.23,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,23995
2090-03-30,56.47,Произвольная программа,lime,62.0,0,2,0,0,0,1,0,0,0,2,0,0,0,1,1,0,0,0,0,0,0,0,2,0,24035


In [100]:
data_unit_train = data_unit.iloc[:len(data_unit)-1]
data_unit_train

,base_score_perf,segment_name,color,school_id,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,days_of_last_tour
date_end,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2089-09-23,49.24,Произвольная программа,lime,62.0,0,2,0,0,1,0,0,0,0,2,0,0,0,2,1,0,0,0,0,0,0,0,0,0,23847
2089-09-23,28.43,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,23847
2089-10-06,42.27,Произвольная программа,lime,62.0,0,2,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,23860
2089-10-06,24.89,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23860
2090-02-18,27.33,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,23995
2090-02-18,53.51,Произвольная программа,lime,62.0,0,2,0,0,0,1,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,2,0,23995
2090-02-18,46.18,Произвольная программа,lime,62.0,0,2,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,2,0,23995
2090-02-18,32.23,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,23995
2090-03-30,56.47,Произвольная программа,lime,62.0,0,2,0,0,0,1,0,0,0,2,0,0,0,1,1,0,0,0,0,0,0,0,2,0,24035


In [101]:
data_unit_train_features = data_unit_train.drop(columns=['1A', '2A', '3A', '1S', '2S', '3S', '4S', '1Lo', '2Lo', '3Lo', '4Lo', '1W', '1T', '2T', '3T', '4T', '1F', '2F', '3F', '4F', '1Lz', '2Lz', '3Lz', '4Lz'], axis=1)
data_unit_train_features

,base_score_perf,segment_name,color,school_id,days_of_last_tour
date_end,,,,,
2089-09-23,49.24,Произвольная программа,lime,62.0,23847
2089-09-23,28.43,Короткая программа,lime,62.0,23847
2089-10-06,42.27,Произвольная программа,lime,62.0,23860
2089-10-06,24.89,Короткая программа,lime,62.0,23860
2090-02-18,27.33,Короткая программа,lime,62.0,23995
2090-02-18,53.51,Произвольная программа,lime,62.0,23995
2090-02-18,46.18,Произвольная программа,lime,62.0,23995
2090-02-18,32.23,Короткая программа,lime,62.0,23995
2090-03-30,56.47,Произвольная программа,lime,62.0,24035


In [102]:
data_unit_train_target = data_unit_train[['1A', '2A', '3A', '1S', '2S', '3S', '4S', '1Lo', '2Lo', '3Lo', '4Lo', '1W', '1T', '2T', '3T', '4T', '1F', '2F', '3F', '4F', '1Lz', '2Lz', '3Lz', '4Lz']]
data_unit_train_target

,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
date_end,,,,,,,,,,,,,,,,,,,,,,,,
2089-09-23,0,2,0,0,1,0,0,0,0,2,0,0,0,2,1,0,0,0,0,0,0,0,0,0
2089-09-23,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
2089-10-06,0,2,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0
2089-10-06,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2090-02-18,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2090-02-18,0,2,0,0,0,1,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,2,0
2090-02-18,0,2,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,2,0
2090-02-18,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2090-03-30,0,2,0,0,0,1,0,0,0,2,0,0,0,1,1,0,0,0,0,0,0,0,2,0


In [103]:
data_unit_train_target['1A']

date_end
2089-09-23    0
2089-09-23    0
2089-10-06    0
2089-10-06    0
2090-02-18    0
2090-02-18    0
2090-02-18    0
2090-02-18    0
2090-03-30    0
2090-03-30    0
2091-01-22    0
2091-01-22    0
2091-02-22    0
2091-03-08    0
2091-03-08    0
2091-04-13    0
2091-04-13    0
2091-04-30    0
2091-04-30    0
2091-09-20    0
2091-09-20    0
2091-10-06    0
2091-10-06    0
2091-10-21    0
2091-10-21    0
2091-12-28    0
2091-12-28    0
2092-03-01    0
Name: 1A, dtype: int64

In [104]:
data_unit_test = data_unit.iloc[len(data_unit)-1:]
data_unit_test

,base_score_perf,segment_name,color,school_id,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,days_of_last_tour
date_end,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2092-03-01,29.05,Короткая программа,lime,62.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,24737


In [105]:
data_unit_test_features =  data_unit_test.drop(columns=['1A', '2A', '3A', '1S', '2S', '3S', '4S', '1Lo', '2Lo', '3Lo', '4Lo', '1W', '1T', '2T', '3T', '4T', '1F', '2F', '3F', '4F', '1Lz', '2Lz', '3Lz', '4Lz'], axis=1)
data_unit_test_features

,base_score_perf,segment_name,color,school_id,days_of_last_tour
date_end,,,,,
2092-03-01,29.05,Короткая программа,lime,62.0,24737


In [106]:
data_unit_test_target = data_unit_test[['1A', '2A', '3A', '1S', '2S', '3S', '4S', '1Lo', '2Lo', '3Lo', '4Lo', '1W', '1T', '2T', '3T', '4T', '1F', '2F', '3F', '4F', '1Lz', '2Lz', '3Lz', '4Lz']]
data_unit_test_target

,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
date_end,,,,,,,,,,,,,,,,,,,,,,,,
2092-03-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [107]:
type(data_unit_test_target)

pandas.core.frame.DataFrame

## Обучаем модель:

In [108]:
category = ['segment_name', 'color', 'school_id']

Циклом перебираем каждый прыжок-таргет (иначе модель не обучается):

In [109]:
predictions = []
for column in data_unit_train_target.columns:
    model = CatBoostClassifier(custom_loss = 'AUC', iterations= 100, depth = 3, learning_rate = 0.3, random_state=42, verbose = 100, allow_const_label=True) # allow_const_label - чтобы CatBoost обучался, даже если значение таргета - константа.
    model.fit(data_unit_train_features, data_unit_train_target[column], cat_features=category) #shuffle=False
    prediction = model.predict(data_unit_test_features).item() # .item() превращает array[[]] в обычное число
    predictions.append(prediction)

predictions

0:	learn: 0.5511705	total: 46.4ms	remaining: 4.6s
99:	learn: 0.0094706	total: 52ms	remaining: 0us
0:	learn: 0.8969745	total: 277us	remaining: 27.5ms
99:	learn: 0.0255901	total: 11.9ms	remaining: 0us
0:	learn: 0.5511705	total: 204us	remaining: 20.3ms
99:	learn: 0.0094706	total: 5.53ms	remaining: 0us
0:	learn: 0.5511705	total: 182us	remaining: 18ms
99:	learn: 0.0094706	total: 6.55ms	remaining: 0us
0:	learn: 0.5859608	total: 264us	remaining: 26.2ms
99:	learn: 0.0414799	total: 10ms	remaining: 0us
0:	learn: 0.5859728	total: 166us	remaining: 16.5ms
99:	learn: 0.0198905	total: 8.93ms	remaining: 0us
0:	learn: 0.5511705	total: 251us	remaining: 24.9ms
99:	learn: 0.0094706	total: 6.76ms	remaining: 0us
0:	learn: 0.5511705	total: 191us	remaining: 19ms
99:	learn: 0.0094706	total: 10.1ms	remaining: 0us
0:	learn: 0.5511705	total: 173us	remaining: 17.2ms
99:	learn: 0.0094706	total: 6.55ms	remaining: 0us
0:	learn: 0.9867325	total: 240us	remaining: 23.8ms
99:	learn: 0.0665603	total: 10.8ms	remaining: 0us

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

In [110]:
type(predictions)

list

In [111]:
predictions_proba = model.predict_proba(data_unit_test_features) # Получим предсказания модели
predictions_proba

array([[0.98692926, 0.01307074]])

In [112]:
predictions

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

In [113]:
len(predictions)

24

In [114]:
predictions = pd.DataFrame([predictions], index = data_unit_test_target.index)
#predictions.columns = ['1A', '2A', '3A', '1S', '2S', '3S', '4S', '1Lo', '2Lo', '3Lo', '4Lo', '1W', '1T', '2T', '3T', '4T', '1F', '2F', '3F', '4F', '1Lz', '2Lz', '3Lz', '4Lz]
predictions

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
date_end,,,,,,,,,,,,,,,,,,,,,,,,
2092-03-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [115]:
data_unit_test_target = data_unit_test_target.reset_index()
del data_unit_test_target['date_end']
data_unit_test_target

,1A,2A,3A,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1W,1T,2T,3T,4T,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [116]:
type(data_unit_test_target)

pandas.core.frame.DataFrame

In [117]:
type(predictions)

pandas.core.frame.DataFrame

In [118]:
print(roc_auc_score(data_unit_test_target, predictions))

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

На одном спортсмене удалось вывести предсказания, допустим удастся и вывести метрику модели по этому одному спортсмену.
Но как вывести метрику по всем спортсменам я пока не поняла. Тупик у меня :(